In [32]:
import pandas as pd
from sklearn.model_selection import train_test_split

from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import accuracy_score, recall_score, precision_score

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import HistGradientBoostingClassifier

In [33]:
df = pd.read_csv("retouren.csv")

**Data Cleaning**

In [34]:
df["user_dob"] = pd.to_datetime(df.user_dob)
df["user_reg_date"] = pd.to_datetime(df.user_reg_date)
df["order_date"] = pd.to_datetime(df.order_date)
df["delivery_date"] = pd.to_datetime(df.delivery_date)

df["age"] = df.order_date.dt.year - df.user_dob.dt.year -((df.order_date.dt.date)<(df.user_dob.dt.date))
df = df.drop(df[df.item_price == 999].index)
df = df.drop(df[df.delivery_date < df.order_date].index)

df.loc[df.item_color == "brwon","item_color"] = "brown"
df.loc[df.item_color == "blau","item_color"] = "blue"
df.loc[df.item_color == "dark oliv","item_color"] = "dark olive"

df = df.drop(df[df.item_color == "?"].index)

df["acc_age"] = df.order_date - df.user_reg_date 
df["acc_age"] = df.acc_age.dt.days

df_mark = df.groupby(["item_id","item_size","item_color"]).agg(maxprice=("item_price","max")).reset_index()
df = pd.merge(df,df_mark)
df["markdown"] = 0
df.loc[df.item_price < df.maxprice,"markdown"] = 1
df.drop(df[df.delivery_date.isna()].index, inplace=True)

**Testdaten**

In [35]:
df_grad = df.copy()

In [36]:
X = df[["item_id", "item_color",	"brand_id",	"item_price",	"user_id",	"user_title",	"user_dob",	"user_state",	"user_reg_date",	"age",	"acc_age",	"markdown"]]
y = df["return"]

In [37]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

**Entscheidungsbaum**

In [38]:
dtc = DecisionTreeClassifier(max_depth= None, min_samples_split=2)
dtc.fit(X_train, y_train)

y_pred_train = dtc.predict(X_train)
y_pred_test = dtc.predict(X_test)

print(f"{'Accuracy Train':17} {accuracy_score(y_train, y_pred_train):10.5f}, {'Accuracy Test':17} {accuracy_score(y_test, y_pred_test):10.5f}")
print(f"{'Recall Train':17} {recall_score(y_train, y_pred_train):10.5f}, {'Recall Test':17} {recall_score(y_test, y_pred_test):10.5f}")
print(f"{'Precision Train':17} {precision_score(y_train, y_pred_train):10.5f}, {'Precision Test':17} {precision_score(y_test, y_pred_test):10.5f}")

nodes = plot_tree(m, rounded=True, filled=True, fontsize=14, class_names=["Not Survived", "Survived"], feature_names=X_train.columns)

ValueError: could not convert string to float: 'black'

**Gradient Boosting Classifier**

In [ ]:
# clf = GradientBoostingClassifier(n_estimators= 100, learning_rate= 1, max_depth= 1, random_state= 0)
# clf.fit(X_train, y_train)
# clf.score(X_test, y_test)